In [ ]:
import numpy as np
import pickle
import tensorflow as tf
import random
import torch
import torch.nn as nn
import torch.optim as optim


In [ ]:

# Specify the path to your pickle file
file_path1 = './samples/training_augmented.pkl'
file_path2 = './samples/testing_augmented.pkl'


with open(file_path1, 'rb') as f:
    # Load the contents of the pickle file
    training_data = pickle.load(f)


with open(file_path2, 'rb') as f:
    # Load the contents of the pickle file
    testing_data = pickle.load(f)

In [ ]:

# Flatten the batches
X_train = []
y_train = []
for batch in training_data:
    for features, label in batch:
        X_train.append(features)
        y_train.append(label)


X_test = []
y_test = []

for features, label in testing_data:
        X_test.append(features)
        y_test.append(label)


X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)




In [ ]:
# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = NeuralNetwork(X_train_tensor.shape[1])

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 70
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size]
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print loss after every epoch (Optional)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the final testing set
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_loss = criterion(final_test_outputs, y_test_tensor)
    predicted_final_test_classes = (final_test_outputs > 0.5).float()
    final_test_accuracy = (predicted_final_test_classes == y_test_tensor).float().mean()

#print("Final Test Loss:", final_test_loss.item())
print("Final Test Accuracy:", final_test_accuracy.item())
